In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt


In [ ]:

# Load and preprocess the data
x = pd.read_csv("Cary_Weather_Data.csv")
x['DATE'] = pd.to_datetime(x[['YEAR', 'MONTH', 'DAY']])
x = x.drop(columns=['YEAR', 'MONTH', 'DAY'])
x = x[['DATE', 'Temperature', 'Humidity', 'Precipitation', 'Wind Speed']]
x.set_index('DATE', inplace=True)



In [ ]:
# Scaling the features
scaler = StandardScaler()
scaled_features = scaler.fit_transform(x[['Temperature', 'Humidity', 'Precipitation', 'Wind Speed']])
scaled_features_df = pd.DataFrame(scaled_features, index=x.index, columns=['Temperature', 'Humidity', 'Precipitation', 'Wind Speed'])

In [ ]:
# Define the create_dataset function
def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)
        ys.append(y.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)




In [ ]:
# Create the dataset with a larger time_steps window
time_steps = 10
X, y = create_dataset(scaled_features_df, scaled_features_df['Temperature'], time_steps)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0, shuffle=False)

# Build the LSTM model
model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    tf.keras.layers.LSTM(50, return_sequences=False),
    tf.keras.layers.Dense(1)
])

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Add callbacks
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model_checkpoint = tf.keras.callbacks.ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)

# Train the model
history = model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=32,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping, model_checkpoint],
    shuffle=False
)

# Plot training and validation loss
plt.figure()
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Predictions
predicted = model.predict(X_val)
predicted = scaler.inverse_transform(np.concatenate((predicted, np.zeros((predicted.shape[0], 3))), axis=1))[:, 0]

# Actual values
actual = scaler.inverse_transform(np.concatenate((y_val.reshape(-1, 1), np.zeros((y_val.shape[0], 3))), axis=1))[:, 0]


In [2]:
%matplotlib qt
# Plot predictions against actual values
plt.figure()
plt.plot(actual, label='Actual Temperature', color='blue')
plt.plot(predicted, label='Predicted Temperature', color='orange')
plt.title('Actual and Predicted Temperatures')
plt.xlabel('Time')
plt.ylabel('Temperature')
plt.legend()
plt.show()
